In [44]:
import pandas as pd
import matplotlib.pyplot as plt
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from re import sub
import pyLDAvis
import nltk
import numpy as np
# import pyLDAvis
import pyLDAvis.gensim_models
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
from wordcloud import WordCloud
from sklearn.utils import resample
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import seaborn as sns

In [63]:
data = pd.read_csv('csvfile_global_twitter_processed.csv')

In [64]:
data.head()

,created_at,source,original_text,polarity,subjectivity,lang,favourite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,LENGTH,preprocessed,clean,sentiment
0,2022-08-03 20:19:13+00:00,"<a href=""http://twitter.com/download/android"" ...",#Pelosi airplane landed safely in #Taiwan 🇹🇼 ...,0.300000,0.203571,en,0,0,DzCritical,318,373,NaN,"[{'text': 'Pelosi', 'indices': [0, 7]}, {'text...",[],Algérie,43,pelosi airplan land safe taiwan 🇹🇼 1 🇨🇳 🇺🇸 pla...,pelosi airplan land safe taiwan 🇹🇼 1 🇨🇳 🇺🇸 pla...,positive
1,2022-08-02 15:24:42+00:00,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Watch the video of the beginning of the Chines...,0.000000,0.000000,en,0,0,toopsat,764,144,False,"[{'text': 'Pelosi', 'indices': [149, 156]}, {'...",[],NaN,28,watch video begin chine bomb taiwan pelosi vis...,watch video begin chine bomb taiwan pelosi vis...,neutral
2,2022-08-02 15:02:35+00:00,"<a href=""http://twitter.com/download/android"" ...",#Pelosi \n#Taipei \n#taiwan\n#XiJinping \n#Chi...,0.000000,0.000000,en,0,0,NassimaLilEmy,64,47,False,"[{'text': 'Pelosi', 'indices': [0, 7]}, {'text...",[],Algérie,12,pelosi taipei taiwan xijinp china verg anoth war,pelosi taipei taiwan xijinp china verg anoth war,neutral
3,2022-08-01 13:51:42+00:00,"<a href=""http://twitter.com/download/android"" ...",#HOBIPALOOZA #LaAcademiaExpulsion #WEURO2022 #...,0.433333,0.733333,en,0,0,d_dhayae,60,463,False,"[{'text': 'HOBIPALOOZA', 'indices': [0, 12]}, ...",[],Chlef,13,hobipalooza laacademiaexpuls weuro2022 jhopeat...,hobipalooza laacademiaexpuls weuro2022 jhopeat...,positive
4,2022-07-31 20:02:20+00:00,"<a href=""http://twitter.com/download/android"" ...",#Pelosi\n#china\nChina Time ✌️ https://t.co/tE...,0.000000,0.000000,en,0,0,Mohamme65404115,39,206,False,"[{'text': 'Pelosi', 'indices': [0, 7]}, {'text...",[],Algérie,6,pelosi china china time ✌️,pelosi china china time ✌️,neutral


In [66]:
wnl = WordNetLemmatizer()

def removePunc(myWord):
    """Function to remove punctuation from string inputs"""
    if myWord is None:
        return myWord
    else:
        return sub('[.:;()/!&-*@$,?^\d+]','',myWord)
        
def removeAscii(myWord):
    """Function to remove ascii from string input"""
    if myWord is None:
        return myWord
    else:
        return str(sub(r'[^\x00-\x7F]+','', myWord.strip()))

def lemmatize(myWord):
    """Function to lemmatize words"""
    if myWord is None:
        return myWord
    else:
        return str(wnl.lemmatize(myWord))

def removeStopWords(myWord):
    """Function to remove stop words"""
    if myWord is None:
        return myWord
    if myWord not in str(stopwords.words('english')):
        return myWord

def removeLinkUser(myWord):
    """Function to remove web addresses and twitter handles"""
    if not myWord.startswith('@') and not myWord.startswith('http'):
        return myWord

def prepText(myWord):
    """Final text pre-processing function"""
    return removeStopWords(
        lemmatize(
            removeAscii(
                removePunc(
                    removeLinkUser(
                        myWord.lower()
                    )
                )
            )
        )
    )

<>:8: DeprecationWarning: invalid escape sequence \d
<>:8: DeprecationWarning: invalid escape sequence \d
<>:8: DeprecationWarning: invalid escape sequence \d
C:\Users\hp\AppData\Local\Temp/ipykernel_28336/3133919804.py:8: DeprecationWarning: invalid escape sequence \d
  return sub('[.:;()/!&-*@$,?^\d+]','',myWord)


In [48]:
def filterTweetList(tweetList):
    """Remove stop words, lemmatize, and clean all tweets"""
    return [[prepText(word) for word
                in tweet.split()
                    if prepText(word) is not None]
                for tweet in tweetList]

In [49]:
cleanKagTweetList = filterTweetList(data['preprocessed'])

In [50]:
cleanKagTweetList[:5]

[['iameztoy',
  'extra',
  'random',
  'imag',
  'let',
  'focu',
  'one',
  'specif',
  'zone',
  'western',
  'coast',
  'longj',
  'district',
  'taichung',
  'citi',
  'ta'],
 ['indopacinfo',
  'china',
  'medium',
  'explain',
  'militari',
  'reason',
  'area',
  'drill',
  'taiwan',
  'strait',
  'read',
  'label',
  'pi'],
 ['china',
  'even',
  'cut',
  'commun',
  'dont',
  'anwer',
  'phonecal',
  'clown',
  'zelenskyyua',
  'enter',
  'stage',
  'ask',
  'xijinp',
  'chang',
  'putin',
  'mind'],
 ['putin',
  'xijinp',
  'tell',
  'friend',
  'taiwan',
  'vassal',
  'state',
  'includ',
  'nuke',
  'much',
  'like',
  'ukrainian',
  'model',
  'warn',
  'take',
  'pelosi',
  'open',
  'china',
  'eye'],
 ['chinauncensor',
  'sorri',
  'think',
  'taiwan',
  'independ',
  'countri',
  'govern',
  'currenc',
  'militari',
  'travel']]

In [51]:
def makeDict(myTweetList):
    """Create dictionary from list of tokenized documents"""
    return corpora.Dictionary(myTweetList)

def makeCorpus(myTweetList,myDict):
    """Create corpus from list of tokenized documents"""
    return [myDict.doc2bow(tweet) for tweet in myTweetList]

def createLDA(myCorpus, myDictionary,myTopics=50,myPasses=10,myIterations=50,myAlpha=0.001):
    """LDA model call function"""
    return models.LdaMulticore(myCorpus, id2word=myDictionary, num_topics=myTopics, passes=myPasses,
    iterations=myIterations,alpha=myAlpha)

In [53]:
"""Create model objects"""
kagDict   = makeDict(cleanKagTweetList)
kagCorpus = makeCorpus(cleanKagTweetList, kagDict)
kagLda = createLDA(kagCorpus, kagDict)

"""Save model objects"""
SaveLoad.save(kagLda,'LDAmodel')
corpora.MmCorpus.serialize('Corpus.mm', kagCorpus)
kagDict.save('Dictionary.dict')

kagLda = SaveLoad.load('LDAmodel')
kagDict = corpora.Dictionary.load('Dictionary.dict')
kagCorpus = corpora.MmCorpus('Corpus.mm')

In [54]:
pyLDAvis.enable_notebook()
ldaViz = pyLDAvis.gensim_models.prepare(kagLda, kagCorpus, kagDict)

C:\Users\hp\anaconda3\envs\Excell-Chatbot\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [55]:
def translateLdaIdx(myLdaModel, myLdaViz):
    """Translate lda model topics to match the topics in pyLDAvis visualization"""
    ldaVizIdx = myLdaViz[0].index
    return list(ldaVizIdx)

In [56]:
newIdx = translateLdaIdx(kagLda,ldaViz)

def createDenseMat(myLdaModel,myCorpus,newIdx):
    """Transform corpus to dataframe with topics matching lda visualization"""
    numTopics = myLdaModel.num_topics
    myDense = corpus2dense(myLdaModel[myCorpus],numTopics)
    myDf = pd.DataFrame(myDense)
    mySortedDf = myDf.transpose()
    mySortedDf = myDf.transpose()[newIdx]
    mySortedDf.columns = ['topic' + str(i + 1) for i in range(numTopics)]
    return mySortedDf

In [57]:
kagDf = createDenseMat(kagLda,kagCorpus,newIdx)

def sortByTopicToIdx(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1):
    """Returns an index of tweets surpassing a topic value threshold"""
    myCleanArray = np.array(cleanedTweetList)
    srtIdx = list(mySortedDf[mySortedDf[myTopic]>myTopicThresh].index)
    return srtIdx

def sortTweetsByIdx(cleanedTweetList,srtIdx):
    """Returns sorted tweets as a list based on a defined sort index"""
    myCleanArray = np.array(cleanedTweetList)
    srtTweets = list(myCleanArray[srtIdx])
    return srtTweets

In [58]:
sortedIdx = sortByTopicToIdx(cleanKagTweetList,kagDf,'topic2',myTopicThresh=0.1)

C:\Users\hp\anaconda3\envs\Excell-Chatbot\lib\site-packages\ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """


In [59]:
# Word cloud for topic 
def makeWordCloud(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1):
    """Create word cloud of tweets passing a given threshold for a given topic"""
    sortedIdx = sortByTopicToIdx(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1)
    mySortedTweets = sortTweetsByIdx(cleanedTweetList,sortedIdx)
    filteredWords = ' '.join([' '.join(string) for string in mySortedTweets])
    myTopicCloud = WordCloud(max_font_size=100,scale=8).generate(filteredWords)
    fig = plt.figure(figsize=(10,10), dpi=1600)
    plt.imshow(myTopicCloud)
    plt.axis("off")
    plt.show()

In [60]:
# It can be seen that most tweeted words are related to war in unkraine and economic crisis.
makeWordCloud(cleanKagTweetList,kagDf,'topic2',myTopicThresh=0.2)

C:\Users\hp\anaconda3\envs\Excell-Chatbot\lib\site-packages\ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """
C:\Users\hp\anaconda3\envs\Excell-Chatbot\lib\site-packages\ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # This is added back by InteractiveShellApp.init_path()
